# 비행 스케쥴 예측 - 지연 시간 예측
# 모델의 정확도(accuracy) 평가(evaluating)
모델을 트레이닝 한 후에는 모델의 accuracy를 파악해야합니다. 모델의 accuracy는, 모델에 의한 예측을 얼마나 확신(confidence) 할 수 있는지에 대한 정보를 제공합니다.

**scitkit-learn** 및 **numpy** 라이브러리는 모델 accuracy 측정에 유용합니다.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
# CSV 파일에서 데이터를 로드
delays_df = pd.read_csv('flight_dataset/Lots_of_flight_data.csv') 
delays_df.head()  # 5건의 데이터만 표시

# 예측을 원하는 데이터는 ARR_DELAY

In [ ]:
delays_df.shape  # 데이터의 구조 파악 - 행열

In [ ]:
# NaN - null 값이 포함된 데이터 확인.
nan_df = delays_df[delays_df.isna().any(axis=1)]
nan_df.head()

In [ ]:
nan_df.shape

In [ ]:
# null값을 제거 - null값이 있으면 트레이닝 과정에서 문제를 유발할 수 있습니다.
delays_df.dropna(inplace=True)


In [ ]:
# Feature column들을 X DataFrame으로 이동
X = delays_df.loc[:,['DISTANCE', 'CRS_ELAPSED_TIME']]

# Labe column을 y DataFrame으로 이동
y = delays_df.loc[:,['ARR_DELAY']] 

# 데이터를 트레이닝 데이터셋과 테스트 데이터셋으로 분리 
X_train, X_test, y_train, y_test = train_test_split(
                                                    X, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=42
                                                   )


In [ ]:
regressor = LinearRegression()     # scikit learn LinearRegression 개체 생성
regressor.fit(X_train, y_train)    # fit 메서드를 사용해 모델 트레이닝 실행



In [ ]:
y_pred = regressor.predict(X_test)

## Accuracy 측정
트레이닝 된 모델이 있으므로 모델의 accuracy를 확인하는데 사용할 수 있는, 여러 메트릭(metric)이 있습니다.

이 메트릭 항목은 모두 수학적 계산을 수행합니다. 여기서 중요한 점은 모든 것을 우리가 직접 계산할 필요가 없다는 것입니다. Scikit-learn과 numpy는 대부분의 작업을 대신 수행합니다.


### Mean Squared Error(평균 제곱 오차 - MSE)
MSE는 결과를 예측할 때 모델이 수행 한 Mean Squared Error입니다.
MSE가 낮을수록 좋은 모델입니다.

MSE는 실제 label 값과 모델에서 predict 한 값 간의 Mean Squared 차이 입니다.

MSE = mean((label값-예측값)^2)

이 계산을 수행하기 위해 label 값과 precit 값을 반복 비교하는 코드를 작성할 수 있지만, 전혀 그럴 필요는 없습니다! **scikit-learn** 라이브러리에서 **mean_squared_error**를 사용하면 됩니다.


In [ ]:
from sklearn import metrics

print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))

In [ ]:
# 추가작업: AutoML - tpop 활용